## Config

In [1]:
# Import dependencies
import tekore as tk
import os
import time
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

In [3]:
### Authenticate Tekore and Authorize App
# Get client token
app_token = tk.request_client_token(CLIENTID, CLIENTSECRET)
# Get a user token; Note the need to ask for write scope in order to create/edit playlists
# This will open a browser window; the user will need to copy the URL from the browser and paste it into the VSCode Command Palette
user_token = tk.prompt_for_user_token(CLIENTID, CLIENTSECRET, REDIRECT_URI, scope=tk.scope.write)
# Create spotify instance
spotify = tk.Spotify(app_token)
# Get the user's spotifyID; The spotify instance will have to reference the user_token to be authorized for this data
with spotify.token_as(user_token):
    userID = spotify.current_user().id

Opening browser for Spotify login...


## Create an empty playlist in the user's account

In [ ]:
# Create a playlist for the user
with spotify.token_as(user_token):
    newPlaylist = spotify.playlist_create(userID, "Summer Playlist Song Candidates", public=False, description='Recent albums from Artist Candidates')
playlistID = newPlaylist.id

## Add some albums to the playlist

In [4]:
# Extract list of songs
songs = pd.read_csv('songs2.csv')
songs

,Unnamed: 0,artistID,artistName,albumID,albumName,albumReleaseDate,albumURI
0,0,12Zk1DFhCbHY6v3xep2ZjI,070 Shake,21ZlnuYMvzSLNOAYWH318k,True love (feat. 070 Shake),2023-04-17,spotify:album:21ZlnuYMvzSLNOAYWH318k
1,1,12Zk1DFhCbHY6v3xep2ZjI,070 Shake,11X8fP4LGagH6xsQI2aqaY,Cocoon (Martin Garrix & Space Ducks Remix),2023-02-10,spotify:album:11X8fP4LGagH6xsQI2aqaY
2,3,5jQsMNuCW0iZeFTz6YUK8K,Pluralone,NaN,NaN,NaN,NaN
3,4,2f0NSj1t2L6JowHINXCFb6,Ruby Red,2FIbisWv6Q7QZVvevvCE3G,Martina Soleil (Living At The Same Time),2023-02-23,spotify:album:2FIbisWv6Q7QZVvevvCE3G


In [ ]:
# Cut the df into chunks
n = 0
while n < len(songs):
    songsToAdd = songs.iloc[n:(n+100)]
    # Add songs to the playlist
    with spotify.token_as(user_token):
        additions = spotify.playlist_add(playlistID, list(songsToAdd['albumURI']), position=None)
    n += 100
    # Wait 2 seconds so we do not exceed our API call allowance
    time.sleep(2)
